# To send personalized table as attachment to ADM vendors

### SOP for using this notebook to send personalized table from a dump file as attachment to ADM vendors
1. Go to https://myaccount.google.com/apppasswords and create new app profile & password associated with your personal flipkart account. Ensure you have 2 step authorization with your flipkart account.
2. The password has to be updated in Step 5 along with your email ID
3. The updated Vendor list along with their email IDs has to be stored in an .xlsx file locally & the path to the file has to be updated in Step 2.
4. The respective vendor files will be processed in a local folder for processing. The folder path has to be updated in Step 4.
5. The file has to be executed in the order mentioned in this file from Step 1 to Step 5.
6. The file contains paths used during testing. Please update before using.

## 1. Install necessary packages in python

In [1]:
# Install packages
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import datetime
from datetime import datetime
from datetime import timedelta
import pandas as pd
import seaborn as sns
import requests
import os
from pathlib import Path

#for sending mail
import smtplib
import imghdr
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import glob

C:\Users\prajwal.d1\AppData\Local\Temp\ipykernel_9660\1044736199.py:16: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


## 2. Read Vendor names & their email IDs from local excel file

In [3]:
# Reading a BGV dump database
Cron_Data = pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Cron_Data_23A.xlsx', dtype=str)
CV_Data = pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Pendency\\CVW1422A.xlsx', dtype=str)
White_Bags2 = pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Pendency\\WBP1A20A.xlsx', dtype=str)
ItTool2 = pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Pendency\\WBIT1A22A.xlsx', dtype=str)
# GTNL= pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Pendency\\gtnl_pendency17Apr.xlsx', dtype=str)
GTNL= pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Pendency\\gtnl_pendency20Apr.xlsx', engine="openpyxl", sheet_name="Sheet1")
GTNLCV = pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Pendency\\GTNLCV11A17A.xlsx', dtype=str)
BRSNR = pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Pendency\\BRSNR_SFR.xlsx', dtype=str)


In [4]:
# Reading a database of vendors & email ids
VendorContact = pd.read_excel(r'C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Vendor_Mail.xlsx', dtype=str)

In [5]:
VendorNames = VendorContact["Vendor Name"].tolist()
VendorMail = VendorContact["Vendor Mail"].tolist()

## 3. Creating file directory & clear old data

In [6]:
# Create a folder and update path here
DownloadPath = "C:\\Users\\prajwal.d1\\Desktop\\Pendency\\Pendency_All"

In [7]:
# Remove all files in the specified folder
os.chdir(f"{DownloadPath}")
all_files = os.listdir()

for f in all_files:
    os.remove(f)

## 4. Processing data for respective vendors

In [8]:
# Create separate files for vendors
attachment_filename_fwd = []

for x in range(len(VendorNames)):
    VendorData_Cron_Data = Cron_Data.loc[Cron_Data['Vendor Name'] == VendorNames[x]]
    VendorData_CV_Data = CV_Data.loc[CV_Data['Vendor Name'] == VendorNames[x]]
    VendorData_White_Bags2 = White_Bags2.loc[White_Bags2['Vendor Name'] == VendorNames[x]]
    VendorData_ItTool2 = ItTool2.loc[ItTool2['Vendor Name'] == VendorNames[x]]
    VendorData_GTNL = GTNL.loc[GTNL['Vendor Name'] == VendorNames[x]]
    VendorData_GTNLCV = GTNLCV.loc[GTNLCV['Vendor Name'] == VendorNames[x]]
    VendorData_BRSNR = BRSNR.loc[BRSNR['Vendor Name'] == VendorNames[x]]
    
    filename=VendorNames[x]+"_Pendency.xlsx"
    with pd.ExcelWriter(filename) as writer:
        VendorData_Cron_Data.to_excel(writer, sheet_name='Pendency_23A',index=False)
        VendorData_CV_Data.to_excel(writer, sheet_name='CV_Data_23A',index=False)
        VendorData_White_Bags2.to_excel(writer, sheet_name='White_Bag_Final_01A20A',index=False)
        VendorData_ItTool2.to_excel(writer, sheet_name='White_Bag_ITTool_1A21A',index=False)
        VendorData_GTNL.to_excel(writer, sheet_name='GTNL_Pendency_20A',index=False)
        VendorData_GTNLCV.to_excel(writer, sheet_name='GTNL_CV_11A17A',index=False)
        VendorData_BRSNR.to_excel(writer, sheet_name='BRSNR_21A',index=False)
        
    attachment_filename_fwd.append(VendorNames[x]+"_Pendency.xlsx")


## 5. Send E-mail to vendor with their respective attachment

In [9]:
# Enter credentials of your account
EMAIL_ADDRESS = 'prajwal.d1@flipkart.com'
EMAIL_PASSWORD = 'hhdmhzadvxnbhqax'

In [10]:
for y in range(len(VendorNames)):
    msg = EmailMessage()
    msg['Subject'] = "RTO_RVP | CV | White Bags | GTNL | BRSNR- 23 April'25 - "+VendorNames[y]
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = VendorMail[y]
    msg.set_content("""Dear Partner,

       Please find the latest data updates below:

  -> RTO/RVP 
    
        -> Ensure that all shipments pending for more than 3 days are cleared by EOD.
        -> For shipments pending for over 7 days, kindly provide a Root Cause Analysis (RCA) without fail.

  -> CV Adherence Update

           Please ensure the CV is updated as per the SOP:
              -> For Flipkart: CV should be done for values greater than 2,000.
              -> For Myntra: CV should be done for values greater than 500.

     ERP CV is not being considered in today's data due to a technical issue.
         
   -> White_Bags
   
         - The latest CL MH White Bag delta data has been attached.
         - Please instruct the hub team and ground team to connect the white bags through the IT tool.
         - IT tool responses are also attached for visibility.
         - Please prioritize these action items and update accordingly.

   -> GTNL Pendency Data.

        - Do go through the data & Kindly Clear the >3 days pendency for avoiding further DN.

   -> BRSNR Pendency Data.

        - BRSNR numbers are rising. Please clear pending cases.
          Ensure all ODHs update the G-Form within TAT to avoid recovery via debit note. 
          Final conclusions will be per FK Zonal, BRM, and Quality remarks.

                      
                      
   Please prioritize these action items and update accordingly.

     Thanks & Regards,
     Prajwal D""")
    attachment_filename_fwd=(VendorContact.loc[VendorContact['Vendor Name'] == VendorNames[y],"Vendor Name"]+"_Pendency.xlsx").tolist()
    files1=[attachment_filename_fwd[0]]
    for file in files1:
        with open(file,'rb') as f:
            file_data1 = f.read()
            file_name1 = f.name
        msg.add_attachment(file_data1, maintype='application', subtype='octet-stream', filename=file_name1)
    #PDF=Temp.loc[Temp['File']=='Gate_pass_Format.pdf',"File"].tolist()
    #files2 =[PDF[0]]
    #for file in files2:
     #   with open(file,'rb') as f:
      #      file_data2 = f.read()
       #     file_name2 = f.name
        #msg.add_attachment(file_data2, maintype='application', subtype='octet-stream', filename=file_name2)
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp: 
        smtp.login(EMAIL_ADDRESS,EMAIL_PASSWORD)
        smtp.send_message(msg)    

In [11]:
FwdDump.head()

NameError: name 'FwdDump' is not defined

In [ ]:
# Remove all files in the specified folder
os.chdir(f"{DownloadPath}")
all_files = os.listdir()
os.getcwd()

In [ ]:
(VendorContact.loc[VendorContact['Vendor Name'] == VendorNames[y],"Vendor Name"]+"_RTO_RVP_UT_28.xlsx")